# Make groups
I am also tracking if the person can come, by default yes unless I am informed otherwise
Will have to figure out a way that Rahel and I are given one spot rather than multiple

In [1]:
import pandas as pd
import numpy as np

people = pd.read_csv('names.csv')
people.head()

,Name,coming,hosts
0,Manthi,1,1
1,Rahel,1,1
2,Manthi/Rahel,1,0
3,SmartyMarty,1,0
4,Sofia ✨,1,0


In [5]:
# how many people are coming
coming_counts = people['coming'].value_counts()
coming_counts

coming
1    25
0     4
Name: count, dtype: int64

In [6]:
MIN_SIZE = 4
PREFERRED_SIZE = 6
MAX_SIZE = 6
SEED = 83

def assign_groups(df, min_size=MIN_SIZE, preferred_size=PREFERRED_SIZE, max_size=MAX_SIZE, seed=SEED):
    N = len(df[df['coming'] == 1])

    if N < min_size:
        raise ValueError("Not enough attendees to form a group.")

    # Shuffle rows
    df = df[df['coming'] == 1].sample(frac=1, random_state=seed).reset_index(drop=True)

    # Start with preferred size groups
    k = N // preferred_size
    r = N % preferred_size
    
    # If remainder is too small (< min_size), redistribute
    if r > 0 and r < min_size:
        # Take one person from each group until remainder reaches min_size
        needed = min_size - r
        if k >= needed:
            k -= 1
            r += preferred_size
        else:
            # Not enough groups to redistribute, adjust strategy
            k = N // min_size
            r = N % min_size
    
    # Build group sizes starting with preferred size
    sizes = [preferred_size] * k
    
    # Distribute remainder
    if r >= min_size:
        sizes.append(r)
    elif r > 0:
        # Distribute remainder across existing groups
        for i in range(r):
            if sizes[i] < max_size:
                sizes[i] += 1

    # Assign group ids
    group_ids = np.empty(N, dtype=int)

    idx = 0
    for group_id, size in enumerate(sizes, start=1):
        group_ids[idx:idx + size] = group_id
        idx += size

    df["group_id"] = group_ids
    return df

In [8]:
# remove hosts from grouping
participants = people[(people['coming'] == 1) & (people['hosts'] == 0)]

In [9]:
output_file = 'grouped_names.csv'
people = assign_groups(participants)
people.to_csv(output_file, index=False)